***Credit Score Card Management*** version by TeetyK

dataset : loan_data_2007_2014 from https://www.kaggle.com/datasets/devanshi23/loan-data-2007-2014/

# import Data & Libraries

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import chi2_contingency

In [26]:
loan_data = pd.read_csv('.\\datasets\\loan_data_2007_2014.csv')

C:\Users\temp67\AppData\Local\Temp\ipykernel_11928\3429229190.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data = pd.read_csv('.\\datasets\\loan_data_2007_2014.csv')


# Data Exploration

In [27]:
loan_data.head(10)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,1075269,1311441,5000,5000,5000.0,36 months,7.90,156.46,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,1069639,1304742,7000,7000,7000.0,60 months,15.96,170.08,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,1072053,1288686,3000,3000,3000.0,36 months,18.64,109.43,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,1071795,1306957,5600,5600,5600.0,60 months,21.28,152.39,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,1071570,1306721,5375,5375,5350.0,60 months,12.69,121.45,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 75 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   466285 non-null  int64  
 1   id                           466285 non-null  int64  
 2   member_id                    466285 non-null  int64  
 3   loan_amnt                    466285 non-null  int64  
 4   funded_amnt                  466285 non-null  int64  
 5   funded_amnt_inv              466285 non-null  float64
 6   term                         466285 non-null  object 
 7   int_rate                     466285 non-null  float64
 8   installment                  466285 non-null  float64
 9   grade                        466285 non-null  object 
 10  sub_grade                    466285 non-null  object 
 11  emp_title                    438697 non-null  object 
 12  emp_length                   445277 non-null  object 
 13 

In [29]:
na_val = loan_data.isnull().mean()
na_val[na_val > 0.8]

mths_since_last_record       0.865666
annual_inc_joint             1.000000
dti_joint                    1.000000
verification_status_joint    1.000000
open_acc_6m                  1.000000
open_il_6m                   1.000000
open_il_12m                  1.000000
open_il_24m                  1.000000
mths_since_rcnt_il           1.000000
total_bal_il                 1.000000
il_util                      1.000000
open_rv_12m                  1.000000
open_rv_24m                  1.000000
max_bal_bc                   1.000000
all_util                     1.000000
inq_fi                       1.000000
total_cu_tl                  1.000000
inq_last_12m                 1.000000
dtype: float64

In [30]:
# ลบ data ที่ missing 80%
loan_data.dropna(thresh = loan_data.shape[0]*0.2,axis=1,inplace=True)
loan_data.drop(columns=['id','member_id','sub_grade',
                'emp_title','url','desc','title','zip_code'
                ,'next_pymnt_d','recoveries','collection_recovery_fee',
                'total_rec_prncp','total_rec_late_fee'],inplace=True)

In [31]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 44 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   466285 non-null  int64  
 1   loan_amnt                    466285 non-null  int64  
 2   funded_amnt                  466285 non-null  int64  
 3   funded_amnt_inv              466285 non-null  float64
 4   term                         466285 non-null  object 
 5   int_rate                     466285 non-null  float64
 6   installment                  466285 non-null  float64
 7   grade                        466285 non-null  object 
 8   emp_length                   445277 non-null  object 
 9   home_ownership               466285 non-null  object 
 10  annual_inc                   466281 non-null  float64
 11  verification_status          466285 non-null  object 
 12  issue_d                      466285 non-null  object 
 13 

# Handle Outiler/Missing

In [32]:
loan_data['loan_status'].value_counts(normalize=True)

loan_status
Current                                                0.480878
Fully Paid                                             0.396193
Charged Off                                            0.091092
Late (31-120 days)                                     0.014798
In Grace Period                                        0.006747
Does not meet the credit policy. Status:Fully Paid     0.004263
Late (16-30 days)                                      0.002612
Default                                                0.001784
Does not meet the credit policy. Status:Charged Off    0.001632
Name: proportion, dtype: float64

In [33]:
# 0 คือลูกค้าไม่ดี 1 คือลูกค้าดี ไม่มีติดหนี้ค้างชำระ
loan_data['good_bad'] = np.where(loan_data.loc[:,'loan_status'].isin(['Charged Off','Default','Late (31-120 days)',
                'Does not meet the credit policy. Status:Charged Off']),0,1)
loan_data.drop(columns=['loan_status'],inplace=True)

In [34]:
loan_data['good_bad'].tail()

466280    1
466281    0
466282    1
466283    1
466284    1
Name: good_bad, dtype: int32

# Split Data

In [102]:
X = loan_data.drop('good_bad',axis=1)
y = loan_data['good_bad']
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify= y)
X_train , X_test = X_train.copy() , X_test.copy() # avoid SettingWithCopyWarning

In [80]:
def emp_length_converter(df,column):
    df[column] = df[column].str.replace('\+ years','',regex=True)
    # df[column] = df[column].str.replace(r'\+','',regex=True)
    df[column] = df[column].str.replace('< 1 year',str(0),regex=True)
    df[column] = df[column].str.replace(' years','',regex=True)
    df[column] = df[column].str.replace(' year','',regex=True)
    df[column] = pd.to_numeric(df[column])
    df[column].fillna(value=0,inplace=True)
emp_length_converter(X_train,'emp_length')

C:\Users\temp67\AppData\Local\Temp\ipykernel_11928\1083151135.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(value=0,inplace=True)


In [81]:
X_train['emp_length'].unique()

array([ 7., 10.,  3.,  4.,  2.,  0.,  1.,  6.,  5.,  8.,  9.])

In [88]:
def date_columns(df,column):
    today_date = pd.to_datetime('2020-08-01')
    df[column] = pd.to_datetime(df[column],format='%b-%y',errors='coerce')
    print(today_date.year , " : " , df[column].dt.year)
    df['mths_since_'+column] = (today_date.year - df[column].dt.year) * 12 + (today_date.month - df[column].dt.month)
    # print(df['mths_since_'+column].describe())
    df['mths_since_'+column] = df['mths_since_'+column].apply(lambda x: df['mths_since_'+column].max() if x < 0 else x)
    df.drop(columns =[column] , inplace=True)
date_columns(X_train,'earliest_cr_line')
date_columns(X_train,'issue_d')
date_columns(X_train,'last_pymnt_d')
date_columns(X_train,'last_credit_pull_d')

2020  :  456615   NaN
451541   NaN
394474   NaN
110294   NaN
139343   NaN
          ..
163633   NaN
399306   NaN
57787    NaN
81641    NaN
38736    NaN
Name: earliest_cr_line, Length: 373028, dtype: float64
2020  :  456615   NaN
451541   NaN
394474   NaN
110294   NaN
139343   NaN
          ..
163633   NaN
399306   NaN
57787    NaN
81641    NaN
38736    NaN
Name: issue_d, Length: 373028, dtype: float64
2020  :  456615   NaN
451541   NaN
394474   NaN
110294   NaN
139343   NaN
          ..
163633   NaN
399306   NaN
57787    NaN
81641    NaN
38736    NaN
Name: last_pymnt_d, Length: 373028, dtype: float64
2020  :  456615   NaN
451541   NaN
394474   NaN
110294   NaN
139343   NaN
          ..
163633   NaN
399306   NaN
57787    NaN
81641    NaN
38736    NaN
Name: last_credit_pull_d, Length: 373028, dtype: float64


In [104]:
# X_train['mths_since_last_credit_pull_d'].unique()
# X_train['last_credit_pull_d'] = pd.to_datetime(X_train['last_credit_pull_d'],format='%b-%y',errors='coerce',infer_datetime_format=True)
# X_train['last_credit_pull_d']
X_train['issue_d']

456615    Jan-14
451541    Jan-14
394474    May-14
110294    Aug-13
139343    May-13
           ...  
163633    Feb-13
399306    May-14
57787     Nov-13
81641     Oct-13
38736     May-08
Name: issue_d, Length: 373028, dtype: object